In [142]:
%matplotlib inline
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib import ticker
plt.rc('axes', axisbelow=True)
import os

plt.rcParams.update({'font.size': 22})

import matplotlib2tikz

In [208]:
DATASET = 'physionet'
TOPOLOGY = "edgeEEGNet"
EXP_ID = '7'
LOG_NOTE = 'cs with net weights from same folds'

BENCHMARK = True
N_ITER = 25

LOG_FOLDER = f'../logs/{DATASET}'
EXP_FOLDER = os.path.join(LOG_FOLDER, TOPOLOGY, f'exp{EXP_ID}')

STATS_FNAME = '{}_class{}_nch{}_split{}'
DATAFILE = os.path.join(EXP_FOLDER, '{}', STATS_FNAME+'{}')

REDUCE_NETW = True
REDUCE_MANUAL = False
RMODE = 'auto' # {'auto', 'dist', 'crow', 'ccprows', 'ccfrows'}
statsfile = os.path.join(EXP_FOLDER, f'stats/{RMODE}_bench.npz')

if DATASET == 'bci':
    num_classes_list = [2,3,4]
    KAPPASCORE = True
else:
    num_classes_list = [2,3,4]
    KAPPASCORE = False
    
n_ch_list = [2, 3, 5, 7, 8, 9, 11, 4, 6, 10, 14, 18, 19, 20, 16, 24, 32, 38, 64]

if DATASET == 'bci':
    ch_dict = {#'auto': [2, 3, 5, 7, 8, 9, 11, 4, 6, 10, 14, 18, 19, 20, 16, 22], 
        #'auto': [2, 3, 5, 7, 8, 9, 11, 4, 22],#, 6, 10, 14, 18, 19, 20, 16, 22], 
        #'auto': [6, 10, 14, 18, 19, 20, 16, 22], 
        'auto': [10,22],
        #'auto': [2, 3, 4, 5, 6, 7, 8, 9, 22],
        #'auto': [10, 11, 14, 16, 18, 19, 20, 22],
               'dist': [], 'crow': [2, 3, 5, 7], 'ccprows': [4, 6], 'ccfrows': [4, 6]}
    TOT_CH = 22
    N_FOLDS = 9
else:
    ch_dict = {
        #'auto': [2, 3, 5, 7, 8, 9, 11, 4, 6, 10, 14, 18, 19, 20, 16, 24, 32, 38, 64],
        #'auto': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 16, 18, 19, 20, 24, 32, 38, 64],
        'auto': [10, 64],
               'dist': [8, 19, 38], 'crow': [2, 3, 5, 7, 9, 11], 
               'ccprows': [4, 6, 10, 14, 18, 20], 'ccfrows': [4, 6, 10, 14, 18, 20]}
    TOT_CH = 64
    N_FOLDS = 5

if RMODE != 'auto':
    manual_ch = ch_dict[RMODE]
    n_ch_set = set(n_ch_list)
    intersection = n_ch_set.intersection(manual_ch)
    n_ch_list = list(intersection)

if KAPPASCORE:
    kappafile = os.path.join(EXP_FOLDER, f'stats/{RMODE}_bench_kappascore.npz')

In [209]:
#statsfile = os.path.join(EXP_FOLDER, f'stats/_bench.npz')

if REDUCE_NETW or REDUCE_MANUAL:
    if TOT_CH in n_ch_list:
        len_nch = len(n_ch_list)
    else:
        len_nch = len(n_ch_list) + 1
else:
    len_nch = len(n_ch_list)

bstats = np.load(statsfile)

if KAPPASCORE:
    kappascore = np.load(kappafile)

train_acc = bstats['train_acc'][:N_ITER, :,:,:]
val_acc = bstats['val_acc'][:N_ITER, :,:,:]
train_locc = bstats['train_loss'][:N_ITER, :,:,:]
val_loss = bstats['val_loss'][:N_ITER, :,:,:]


if np.sum(val_acc[-1,:,:,:]) == 0:
    print(f'ATTENTION! benchmark {N_ITER} runs not complete')

mean_val_acc = np.mean(val_acc, axis=(0,3))

for i, num_classes in enumerate(num_classes_list):
    print("accuracy num_classes: ", num_classes)
    for c, acc in zip(ch_dict[RMODE], mean_val_acc[i,:]*100):
        print(c, acc)
        
if KAPPASCORE:
    kappa = kappascore['kappascore'][:N_ITER,:,:,:]
    mean_kappa = np.mean(kappa, axis=(0,3))

    for i, num_classes in enumerate(num_classes_list):
        print("kappascore num classes", num_classes)
        for c, k in zip(ch_dict[RMODE], mean_kappa[i,:]):
            print(c, k)

accuracy num_classes:  2
10 82.26757370506554
64 83.25170068621907
accuracy num_classes:  3
10 70.633408927503
64 73.18820862227646
accuracy num_classes:  4
10 62.095238095238095
64 63.718820861678005


In [183]:
print(np.mean(val_acc, axis=0))
print('sorted', np.sort(np.mean(val_acc, axis=0)))

[[[0.83219955 0.82426304 0.85600907 0.78798186 0.82993197]
  [0.         0.         0.         0.         0.        ]]

 [[0.73998488 0.71957672 0.76870748 0.71050642 0.73091459]
  [0.718065   0.77097506 0.81632653 0.72411187 0.72033258]]

 [[0.64455782 0.62414966 0.67687075 0.57709751 0.62528345]
  [0.65929705 0.66043084 0.6978458  0.6207483  0.62868481]]]
sorted [[[0.78798186 0.82426304 0.82993197 0.83219955 0.85600907]
  [0.         0.         0.         0.         0.        ]]

 [[0.71050642 0.71957672 0.73091459 0.73998488 0.76870748]
  [0.718065   0.72033258 0.72411187 0.77097506 0.81632653]]

 [[0.57709751 0.62414966 0.62528345 0.64455782 0.67687075]
  [0.6207483  0.62868481 0.65929705 0.66043084 0.6978458 ]]]


In [78]:
print(np.mean(kappa, axis=0))
print('sorted', np.sort(np.mean(kappa, axis=0)))

[[[0.45753939 0.19907628 0.54884783 0.31967282 0.2857096  0.17033879
   0.39297394 0.40760843 0.44064219]
  [0.56292565 0.31755604 0.67794245 0.3720706  0.38291811 0.23444516
   0.4310017  0.57059903 0.53568277]
  [0.69742784 0.39509994 0.81211142 0.46644761 0.494041   0.31381373
   0.52699327 0.66478996 0.6013558 ]
  [0.7350361  0.40248032 0.84638205 0.48541213 0.52211858 0.35525692
   0.61889084 0.71885832 0.6404668 ]
  [0.75117025 0.42511647 0.84831966 0.50987053 0.53918464 0.35714042
   0.60887224 0.73978511 0.65525234]
  [0.74854211 0.42409576 0.8375324  0.49820336 0.55202568 0.38421392
   0.67227901 0.74274165 0.65231349]
  [0.75210533 0.41086003 0.85671464 0.52107707 0.54471849 0.3506175
   0.73299024 0.75390829 0.67137343]
  [0.634778   0.32615556 0.76619766 0.42143878 0.43082466 0.29399167
   0.52976543 0.65101041 0.57390756]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.        ]]]
sorted [[[0.17033879 0.19907628 0.2857096  0.31967282

In [85]:
sort_i = np.argsort(ch_dict[RMODE])
sorted_ch = np.sort(ch_dict[RMODE])
sorted_val = mean_val_acc[0, sort_i]

print(ch_dict[RMODE], '\n', sorted_ch, '\n', sort_i, '\n', sorted_val)

[6, 10, 14, 18, 19, 20, 16, 22] 
 [ 6 10 14 16 18 19 20 22] 
 [0 1 2 6 3 4 5 7] 
 [0.68010199 0.71870843 0.72477177 0.72432355 0.72905467 0.72454137
 0.72399456 0.        ]


In [86]:
for i, num_classes in enumerate(num_classes_list):
    print("num_classes: ", num_classes)
    for c, acc in zip(sorted_ch, sorted_val*100):
        print(c, acc)

num_classes:  4
6 68.01019907576216
10 71.87084262331544
14 72.47717741339584
16 72.43235489974379
18 72.90546714062523
19 72.45413669999346
20 72.39945570503296
22 0.0
